# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

In [11]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

#### b) Create the builder function for step 1.

In [4]:
label_randomly <- function(n_points, n_clusters){
  sample(((1:n_points) %% n_clusters)+1, n_points, replace=F)
}

#### c) Create the builder function for step 2.

In [5]:
get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [6]:
assign_cluster <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped)^2
  all_distances <- rowSums(diff_squared)
  tibble(dii=dii, mii=mii, distance=all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

#### e) Create the builder function for step 4.

In [7]:
kmeans_done <- function(old_means, new_means, eps=1e-6){
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)
  m <- mean(sqrt(rowSums((om - nm)^2)))
  if(m < eps) TRUE else FALSE
}

#### f) Combine them all into your own `kmeans` function.

In [13]:
mykmeans <- function(data, n_clusters, eps=1e-6, max_it = 1000, verbose = FALSE){
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  it <- 0
  while(!done & it < max_it){
    labels <- assign_cluster(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if(kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
      it <- it + 1
      if(verbose){
        cat(sprintf("%d\n", it))
      }
    }
  }
  list(labels=labels, means=new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [9]:
data <- read.csv('voltages_df.csv')
head(data, 10)

,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,X9.03614457831325,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.031463,1.104665,0.8982475,0.4142208,-1.1490888,-1.078510,-1.002401,-0.9182083,-0.8215574,-0.7023741,⋯,-0.73927028,-0.7633694,-0.7792297,-0.78443398,-0.7779820,-0.7608812,-0.7369830,-0.7138199,-0.7014771,-0.7056029
2,-1.031463,1.246157,1.0948587,0.9039343,0.4654410,-1.160496,-1.112005,-1.0721319,-1.0385633,-1.0075872,⋯,-0.88599643,-0.8511675,-0.8064307,-0.75345581,-0.6954785,-0.6404759,-0.6105817,-0.6348313,-0.6767121,-0.7140939
3,-1.031463,1.216111,1.0557873,0.8417629,-0.5636836,-1.147653,-1.101783,-1.0645681,-1.0336197,-1.0051885,⋯,-0.95035091,-0.9122991,-0.8625269,-0.80161422,-0.7306757,-0.6527186,-0.5812047,-0.5875560,-0.6768023,-0.7206992
4,-1.031463,1.166244,0.9899628,0.7230858,-1.1806746,-1.125106,-1.077167,-1.0370309,-1.0027385,-0.9709488,⋯,-0.94985086,-0.9236047,-0.8896604,-0.85021204,-0.8086367,-0.7700917,-0.7418958,-0.7315532,-0.7409824,-0.7644406
5,-1.031463,1.230222,1.0746700,0.8733880,0.2116394,-1.153728,-1.106832,-1.0691075,-1.0383350,-1.0108187,⋯,-0.87101657,-0.8237315,-0.7590005,-0.66985818,-0.5061566,1.0975578,0.9348933,0.6673692,-1.1669718,-1.1047735
6,-1.031463,1.257650,1.1112886,0.9322788,0.6045620,-1.166325,-1.113488,-1.0696859,-1.0332517,-1.0009558,⋯,-0.90923416,-0.8715416,-0.8213803,-0.75895965,-0.6842115,-0.5958772,-0.4766488,1.1008087,0.9169321,0.5137345
7,-1.031463,1.316138,1.1695503,0.9890025,0.6965879,-1.179714,-1.122019,-1.0612423,-0.9951141,-0.9213625,⋯,-0.09070482,1.0328294,0.8446656,0.19768020,-1.1385099,-1.0777650,-1.0227513,-0.9710004,-0.9192536,-0.8640508
8,-1.031463,1.304807,1.1561862,0.9715807,0.6558179,-1.175046,-1.117527,-1.0575812,-0.9935313,-0.9240551,⋯,0.19083663,1.0250044,0.8318109,-0.08138448,-1.1368559,-1.0793894,-1.0280343,-0.9797811,-0.9308302,-0.8772506
9,-1.031463,1.255708,1.1069568,0.9217357,0.5472037,-1.165061,-1.116478,-1.0768723,-1.0440300,-1.0143181,⋯,-0.95297338,-0.9263942,-0.8914488,-0.85031607,-0.8063880,-0.7650622,-0.7345009,-0.7237009,-0.7350932,-0.7615456


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [16]:
results <- mykmeans(data, 3)
results$labels
results$means

[1] 2 3 3 3 3 3 1 1 3 2 2 2 1 2 3 2 2 2 2 2 3 3 3 3 3 2 2 2 1 1 1 1 3 1 1 3 3
 [38] 2 1 3 2 2 2 1 3 1 1 3 1 1 3 1 2 2 1 2 3 2 1 3 2 2 3 2 1 1 3 1 2 1 2 3 1 1
 [75] 3 3 2 1 1 1 1 1 3 1 1 3 3 3 2 1 1 1 3 1 3 3 1 3 3 1 1 1 3 1 2 2 1 2 2 3 2
[112] 2 3 1 2 1 2 2 3 3 3 3 2 2 1 3 3 1 2 1 3 2 3 1 2 3 3 3 2 1 2 3 3 1 2 2 3 2
[149] 2 1 2 1 3 3 1 2 1 1 3 2 1 1 2 3 2 1 3 2 3 2 2 1 3 2 1 1 3 1 2 3 1 3 1 2 1
[186] 3 3 2 3 2 2 3 2 2 1 1 2 3 2 2 1 2 1 1 3 2 3 1 3 2 3 1 2 1 1 3 1 1 3 3 3 3
[223] 3 3 3 1 1 1 1 2 3 2 1 3 3 1 1 2 2 1 1 2 1 1 3 1 2 1 1 3 1 3 2 1 3 1 1 2 2
[260] 2 3 3 1 2 3 3 3 1 1 1 2 3 3 3 1 2 1 1 3 3 1 2 1 2 2 3 1 3 3 1 1 2 1 1 2 3
[297] 2 1 1 3 3 1 2 2 2 1 2 3 3 3 2 2 2 2 2 2 3 2 3 2 2 1 3 1 2 2 2 2 3 2 3 2 1
[334] 2 3 3 1 1 2 3 1 3 1 1 2 1 3 1 2 2 1 3 3 3 2 1 3 2 1 1 3 3 3 3 2 1 2 3 2 2
[371] 2 2 2 3 3 1 1 2 2 2 2 1 1 2 2 1 1 3 3 1 2 3 3 3 3 1 3 3 1 2 2 2 2 3 3 2 2
[408] 1 1 1 3 1 1 2 3 3 3 1 3 3 3 2 2 3 2 1 3 3 3 3 1 1 3 1 2 3 2 2 2 1 1 1 1 2
[445] 3 2 2 2 1 3 2 2 2 3 3 2 3 3 2 1 3 3 2 1 3 3 3 2 1 1 2 1 2 3 3 1 2 1 2 3 3
[482] 1 2 1 1 1 3 1 1 1 1 1 3 2 2 1 1 2 3 3 3 2 2 2 3 3 1 1 2 1 2 3 3 1 1 2 1 1
[519] 3 2 2 2 3 1 2 3 1 1 2 3 1 1 1 1 2 2 3 2 2 3 3 3 3 1 2 2 1 2 2 2 3 3 2 3 1
[556] 1 3 1 1 1 3 1 1 3 1 1 3 1 1 2 1 3 1 3 1 3 3 3 2 1 1 1 1 2 3 1 1 1 3 1 1 3
[593] 3 3 1 2 3 3 2 3 1 3 3 1 2 2 1 1 1 3 1 2 3 2 1 3 2 3 3 2 1 3 2 1 1 2 3 2 1
[630] 3 2 1 2 3 3 2 3 2 2 2 2 2 3 3 2 1 2 1 1 2 1 3 1 2 1 2 2 2 3 2 2 3 3 2 2 3
[667] 2 3 1 1 3 2 3 2 2 1 3 3 2 1 2 3 1 1 2 2 3 2 3 3 1 1 1 1 3 3 3 2 1 3 2 1 1
[704] 2 2 1 3 1 1 2 3 2 1 3 2 3 3 2 3 3 2 2 3 3 2 3 1 2 2 2 2 2 3 3 2 2 3 3 2 3
[741] 2 3 1 2 2 2 3 2 1 3 3 2 2 1 2 2 1 2 1 1 2 2 3 1 2 3 1 3 3 2 1 3 1 2 3 2 2
[778] 1 2 1 3 2 2 3 1 3 2 3 2 1 2 2 3 3 3 2 3 2 1 3 3 3 3 2 2 1 3 2 2 2 2 1 1 2
[815] 1 2 3 2 3 1 3 3 3 3 3 1 1 3 1 1 1 1 1 1 3 2 2 3 1 1 2 3 1 1 1 1 2 2 3 1 3
[852] 2 1 1 1 1 2 2 1 1 1 1 1 2 3 1 3 2 3 3 1 2 1 1 2 2 3 2 2 2 1 3 1 3 3 2 2 1
[889] 1 3 2 1 1 3 3 3 3 2 1 1

label__,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.031463,1.3093239,1.1616772,0.9787498,0.6481497,-1.168610,-1.1196122,-1.0590962,-0.9943176,⋯,0.3364266,0.8337474,0.7125412,-0.2659209,-1.0409179,-1.0587745,-1.01359887,-0.96467777,-0.9151047,-0.8610245
2,-1.031463,0.9381238,0.7619864,0.3631543,-1.1179412,-1.051145,-0.9766807,-0.8694758,-0.6892375,⋯,-0.7900387,-0.8070676,-0.8182598,-0.8207339,-0.8132928,-0.7969549,-0.77567272,-0.75689256,-0.7496483,-0.7570393
3,-1.031463,1.2439759,1.0924697,0.9004440,0.3011754,-1.159714,-1.1098127,-1.0685484,-1.0338649,⋯,-0.9107472,-0.8732292,-0.8234477,-0.7607812,-0.6682618,-0.3380864,-0.04693168,0.02820486,-0.4113500,-0.8115784


#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [24]:
df <- data %>% as.matrix()
result_r <- kmeans(df, 3)
result_r$cluster
result_r$centers

[1] 3 1 1 1 1 1 2 2 1 3 3 3 2 3 1 3 3 3 3 3 1 1 1 1 1 3 3 3 2 2 2 2 1 2 2 1 1
 [38] 3 2 1 3 3 3 2 1 2 2 1 2 2 1 2 3 3 2 3 1 3 2 1 3 3 1 3 2 2 1 2 3 2 3 1 2 2
 [75] 1 1 3 2 2 2 2 2 1 2 2 1 1 1 3 2 2 2 1 2 1 1 2 1 1 2 2 2 1 2 3 3 2 3 3 1 3
[112] 3 1 2 3 2 3 3 1 1 1 1 3 3 2 1 1 2 3 2 1 3 1 2 3 1 1 1 3 2 3 1 1 2 3 3 1 3
[149] 3 2 3 2 1 1 2 3 2 2 1 3 2 2 3 1 3 2 1 3 1 3 3 2 1 3 2 2 1 2 3 1 2 1 2 3 2
[186] 1 1 3 1 3 3 1 3 3 2 2 3 1 3 3 2 3 2 2 1 3 1 2 1 3 1 2 3 2 2 1 2 2 1 1 1 1
[223] 1 1 1 2 2 2 2 3 1 3 2 1 1 2 2 3 3 2 2 3 2 2 1 2 3 2 2 1 2 1 3 2 1 2 2 3 3
[260] 3 1 1 2 3 1 1 1 2 2 2 3 1 1 1 2 3 2 2 1 1 2 3 2 3 3 1 2 1 1 2 2 3 2 2 3 1
[297] 3 2 2 1 1 2 3 3 3 2 3 1 1 1 3 3 3 3 3 3 1 3 1 3 3 2 1 2 3 3 3 3 1 3 1 3 2
[334] 3 1 1 2 2 3 1 2 1 2 2 3 2 1 2 3 3 2 1 1 1 3 2 1 3 2 2 1 1 1 1 3 2 3 1 3 3
[371] 3 3 3 1 1 2 2 3 3 3 3 2 2 3 3 2 2 1 1 2 3 1 1 1 1 2 1 1 2 3 3 3 3 1 1 3 3
[408] 2 2 2 1 2 2 3 1 1 1 2 1 1 1 3 3 1 3 2 1 1 1 1 2 2 1 2 3 1 3 3 3 2 2 2 2 3
[445] 1 3 3 3 2 1 3 3 3 1 1 3 1 1 3 2 1 1 3 2 1 1 1 3 2 2 3 2 3 1 1 2 3 2 3 1 1
[482] 2 3 2 2 2 1 2 2 2 2 2 1 3 3 2 2 3 1 1 1 3 3 3 1 1 2 2 3 2 3 1 1 2 2 3 2 2
[519] 1 3 3 3 1 2 3 1 2 2 3 1 2 2 2 2 3 3 1 3 3 1 1 1 1 2 3 3 2 3 3 3 1 1 3 1 2
[556] 2 1 2 2 2 1 2 2 1 2 2 1 2 2 3 2 1 2 1 2 1 1 1 3 2 2 2 2 3 1 2 2 2 1 2 2 1
[593] 1 1 2 3 1 1 3 1 2 1 1 2 3 3 2 2 2 1 2 3 1 3 2 1 3 1 1 3 2 1 3 2 2 3 1 3 2
[630] 1 3 2 3 1 1 3 1 3 3 3 3 3 1 1 3 2 3 2 2 3 2 1 2 3 2 3 3 3 1 3 3 1 1 3 3 1
[667] 3 1 2 2 1 3 1 3 3 2 1 1 3 2 3 1 2 2 3 3 1 3 1 1 2 2 2 2 1 1 1 3 2 1 3 2 2
[704] 3 3 2 1 2 2 3 1 3 2 1 3 1 1 3 1 1 3 3 1 1 3 1 2 3 3 3 3 3 1 1 3 3 1 1 3 1
[741] 3 1 2 3 3 3 1 3 2 1 1 3 3 2 3 3 2 3 2 2 3 3 1 2 3 1 2 1 1 3 2 1 2 3 1 3 3
[778] 2 3 2 1 3 3 1 2 1 3 1 3 2 3 3 1 1 1 3 1 3 2 1 1 1 1 3 3 2 1 3 3 3 3 2 2 3
[815] 2 3 1 3 1 2 1 1 1 1 1 2 2 1 2 2 2 2 2 2 1 3 3 1 2 2 3 1 2 2 2 2 3 3 1 2 1
[852] 3 2 2 2 2 3 3 2 2 2 2 2 3 1 2 1 3 1 1 2 3 2 2 3 3 1 3 3 3 2 1 2 1 1 3 3 2
[889] 2 1 3 2 2 1 1 1 1 3 2 2

,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,X9.03614457831325,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
1,-1.031463,1.2439759,1.0924697,0.9004440,0.3011754,-1.159714,-1.1098127,-1.0685484,-1.0338649,-1.0022396,⋯,-0.9107472,-0.8732292,-0.8234477,-0.7607812,-0.6682618,-0.3380864,-0.04693168,0.02820486,-0.4113500,-0.8115784
2,-1.031463,1.3093239,1.1616772,0.9787498,0.6481497,-1.168610,-1.1196122,-1.0590962,-0.9943176,-0.9237437,⋯,0.3364266,0.8337474,0.7125412,-0.2659209,-1.0409179,-1.0587745,-1.01359887,-0.96467777,-0.9151047,-0.8610245
3,-1.031463,0.9381238,0.7619864,0.3631543,-1.1179412,-1.051145,-0.9766807,-0.8694758,-0.6892375,-0.5661321,⋯,-0.7900387,-0.8070676,-0.8182598,-0.8207339,-0.8132928,-0.7969549,-0.77567272,-0.75689256,-0.7496483,-0.7570393


#### d) Are your labels/clusters the same? If not, why? Are your means the same?

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

#### c) State which (for loops or vectorizing) is more efficient and why.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

## Question 5
#### What assumption do Guassian mixture models make?

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

## Question 7
#### Define the gap statistic method. What do we use it for?